In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import os
print(os.listdir("../input"))

In [ ]:
from fastai.text import *
from fastai.imports import *
from fastai.text import *
from fastai import *

In [ ]:
test = pd.read_csv('/kaggle/input/Test.csv')
train = pd.read_csv('/kaggle/input/Train.csv')

In [ ]:
train.head(5)

In [ ]:
train.isnull().sum()
#test.isnull().sum()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train['safe_text'] = train['safe_text'].apply(clean_text)
test['safe_text'] = test['safe_text'].apply(clean_text)

In [ ]:
import seaborn as sns

sns.countplot(x='label', data=train)

In [ ]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [ ]:
path = Path('../input/')
path.ls()

In [ ]:

from sklearn.metrics import accuracy_score 
y_pred_totcb = []
from sklearn.model_selection import KFold, RepeatedKFold
fold = KFold(n_splits=15, shuffle=True, random_state=0)
i=1

for train_index, test_index in fold.split(train):
    
    train_df = train.iloc[train_index]
    valid_df = train.iloc[test_index]

    random_seed(10)
    
    data_lm = TextLMDataBunch.from_df(Path(path), train_df, valid_df, test, text_cols=[0], bs=32)
    data_clas = TextClasDataBunch.from_df(Path(path), train_df, valid_df, test, text_cols=[0], label_cols=1, bs=32)
    
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.4, model_dir='/tmp/model/')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.unfreeze()
    learn.fit_one_cycle(9, 1e-3, moms=(0.8,0.7))
    learn.save_encoder('model_enc')
    
    learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.4, model_dir='/tmp/model/')
    learn.load_encoder('model_enc')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.freeze_to(-2)
    learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
    learn.freeze_to(-3)
    learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
    learn.unfreeze()
    learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))
   
    log_preds, test_labels = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
    preds = np.argmax(log_preds, 1)
    y_pred_totcb.append(preds)
    print(f'fold {i} completed')
    i = i+1

In [ ]:
y_pred_totcb

In [ ]:

df = pd.DataFrame()
col_name = 'SECTION_' + str(i)
df[col_name] =  y_pred_totcb[0]

In [ ]:
df = pd.DataFrame()
for i in range(2):
    col_name = 'SECTION_' + str(i)
    df[col_name] =  y_pred_totcb[i]

In [ ]:

df.shape

In [ ]:
sub = pd.DataFrame()
sub['SECTION'] = df.mode(axis=1)[0]
sub.tail()

In [ ]:
sub['SECTION'].value_counts()

In [ ]:
submission = pd.DataFrame({
        "labels": sub['SECTION']
    })
submission.to_csv('./z1.csv', index=False)
print(submission)